In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the test data
X_test = np.load('../data/splits/test_images.npy')
y_test = np.load('../data/splits/test_labels.npy')

# Convert grayscale to 3-channel images for pre-trained models
X_test_rgb = np.repeat(X_test, 3, axis=-1)

# Load the models
custom_cnn = tf.keras.models.load_model('../models/cnn/custom_cnn.keras')
vgg16 = tf.keras.models.load_model('../models/pretrained/vgg16.keras')
resnet50 = tf.keras.models.load_model('../models/pretrained/resnet50.keras')
inceptionv3 = tf.keras.models.load_model('../models/pretrained/inceptionv3.keras')

# Generate predictions
custom_preds = custom_cnn.predict(X_test).flatten()
vgg16_preds = vgg16.predict(X_test_rgb).flatten()
resnet50_preds = resnet50.predict(X_test_rgb).flatten()
inceptionv3_preds = inceptionv3.predict(X_test_rgb).flatten()

# Convert predictions to binary labels
def binarize_preds(preds, threshold=0.5):
    return (preds > threshold).astype(int)

custom_binary = binarize_preds(custom_preds)
vgg16_binary = binarize_preds(vgg16_preds)
resnet50_binary = binarize_preds(resnet50_preds)
inceptionv3_binary = binarize_preds(inceptionv3_preds)

# Ensemble method: Majority Voting
def majority_voting(*args):
    return (np.sum(args, axis=0) >= (len(args) / 2)).astype(int)

ensemble_preds = majority_voting(custom_binary, vgg16_binary, resnet50_binary, inceptionv3_binary)

# Evaluate the ensemble model
accuracy = accuracy_score(y_test, ensemble_preds)
conf_matrix = confusion_matrix(y_test, ensemble_preds)
class_report = classification_report(y_test, ensemble_preds)

print(f"Ensemble Model Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")
